In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates
import ctapipe
from lstchain.io import replace_config, standard_config
import h5py
from astropy.io import fits
import json
muon_rings = [434, 663, 949, 1295, 1355, 1565, 1777, 1933, 3067, 3179, 3270, 3314,3346, 3692, 4076,  5016, 6853, 7723, 7919, 8122, 8605, 9047, 9157, 9239, 9774,  9910, 10486, 10491, 11600, 11781, 12038, 12506, 12969, 13004, 13101] 
muon_rings_datacheck = [54110, 55656, 55942, 57623, 58136, 58798, 59194, 59570, 60407, 60422, 62395, 63033, 63412, 64971, 65272, 66251, 66737, 67143, 67175, 67772, 68483, 68495, 68755, 68902, 69300, 69891, 70855, 72398, 73259, 73261, 73725, 76581, 76871, 77277, 80267, 80982, 81229, 81606, 82140, 83970, 84318, 85580, 87424, 87427, 87921, 88388, 89632, 91351, 93414, 93971, 94651, 95006, 96661, 97072, 97242, 97335, 97713, 97785, 97891, 99847, 100267, 101084, 101156, 101606, 103028, 103475, 105020, 105713]
flagged_rings_sim = [5333507, 1673009, 1808303, 3362305, 2835508, 4026706, 159503, 4854202, 5464504, 2439702, 1402200, 3456509, 3547004, 2229705, 22604, 5573308, 3489206, 417000, 5895006, 1836008, 1794401, 593906, 3280901, 1114103, 5920700, 5521903, 5240406, 5678007, 918003, 5858809, 4315801, 3547006, 729206, 5412204, 759808, 3386907, 2597005, 3539901, 4125904, 1073405, 3662503, 846606, 2845905, 5139408, 5097001, 3248609, 2257305, 5559707, 1775404, 5586609, 5387503, 336801, 3611003, 4504102, 1556201, 3742006, 55203, 377809, 5303909, 3393108, 5854105, 3850800, 5240903, 5120108, 4330605, 4006201, 4752000, 817501, 1096109, 5562603, 3289408, 2057308, 1646106, 2736603, 220203, 2095301, 243205, 250008, 728702, 1177701, 402700, 4589808, 1063003, 3804102, 1088507, 5638409, 1824304, 2482107, 2619809, 4647903, 5864908, 4805401, 2936707, 996201]
len(muon_rings)
import os
import pickle


In [ ]:
# Треба завантажити конфіг в івент соурс
filename = f'/Users/vdk/CTA/RealLST/typical_run_data/LST-1.1.Run14948.0001.fits.fz'

int_cut = 0.3
min_pixel = 30

config_filename = '/Users/vdk/CTA/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'
with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)

# muon_processor_config = Config({"MuonProcessor": 
#                                 {"RingQuery": 
#                                  {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
#                                                         ["ring_containment", "parameters.containment > 0.5"],
#                                                         ["ring_completeness", "parameters.completeness > 0.5"]]}}})

# muon_processor_config_2 = Config({"MuonProcessor":
#                                 {"ImageParameterQuery" :
#                                  {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                         ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/CTA/RealLST/typical_run_data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/CTA/RealLST/typical_run_data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/CTA/RealLST/typical_run_data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/CTA/RealLST/typical_run_data/time_calibration.Run08349.0000.h5'
      }})

# config = replace_config(standard_config, muon_processor_config)
# config = replace_config(config, muon_processor_config_2)
#config = replace_config(config, custom_config)


In [ ]:
source = EventSource('/Users/vdk/Software/ctasoft/calibpipe/src/calibpipe/tests/data/throughput/Dummy100_NSB.simtel.gz')
#source = EventSource('/Users/vdk/CTA/GeneveWork/muon_simpaper/data/run401_muon_lst.simtel.gz')
event_iterator = iter(source)
event = next(event_iterator)
for i,event in enumerate(event_iterator):    
    for pix_id in range(0, 1855):
        plt.plot(
            event.r0.tel[1].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle="steps"
        )
        
    plt.title('LG')
    plt.show()
    for pix_id in range(0, 1855):
        plt.plot(
            event.r0.tel[1].waveform[1][pix_id], label="pix {}".format(pix_id), drawstyle="steps"
        )
    plt.title(f'HG {i}')
    plt.show()
    if i > 100:
        plt.close()
        break

In [ ]:
source = EventSource(filename, config = Config(cfg))

event_iterator = iter(source)
event = next(event_iterator)

event.index.event_id

In [ ]:
event.r0.tel[1]

In [ ]:
if (len(event.trigger.tels_with_trigger) == 1) and (event.trigger.tels_with_trigger[0] == 1):
    filename = f'/fefs/aswg/workspace/vadym.voitsekhovskyi/1LST/waveforms_1LST/R1waveform{event.index.event_id}.txt'
    np.save(filename, event.r1.tel[1].waveform)

### Process events and create dict with muon rings

In [ ]:
print(event.mon.tel[1].calibration.unusable_pixels)


In [ ]:
#event = next(event_iterator)
# There are 13247 events

cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/CTA/RealLST/typical_run_data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/CTA/RealLST/typical_run_data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/CTA/RealLST/typical_run_data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/CTA/RealLST/typical_run_data/time_calibration.Run08349.0000.h5'
      }})
source = EventSource(filename, config = Config(cfg))

event_iterator = iter(source)
config = replace_config(standard_config, muon_processor_config)
config = replace_config(config, muon_processor_config_2)
config = replace_config(config, custom_config)

optical_efficiency_list = []
muon_arr = []
muon_list = {}
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray, config = Config(config))
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray, config = Config(config))
n_pixels = []
arr_max_muons = []
arg_max_muons = []

arr_max_muons_mask = []
arg_max_muons_mask = []

arr_max_protons = []
arg_max_protons = []

arr_max_protons_mask = []
arg_max_protons_mask = []
counter = 0
for iter_number, event in enumerate(event_iterator):
   #print(iter_number)
   #calib(event)
   #image_processor(event)
   #muon_processor(event)
   #print("n pixels = ", event.dl1.tel[1].parameters.morphology.n_pixels)
   #n_pixels.append(event.dl1.tel[1].parameters.morphology.n_pixels)
   #pixel_mask = np.array(np.max(event.r1.tel[1].waveform, axis = 1) > 2.5, dtype = bool)
   if event.index.event_id in muon_rings_datacheck:
      counter += 1
      calib(event)
      image_processor(event)
      muon_processor(event)
      plt.figure(figsize = (9,6))
      camgeom = source.subarray.tel[1].camera.geometry
      title = f'iter_number = {iter_number}, event_id = {event.index.event_id}, opt_eff = {round(event.muon.tel[1].efficiency.optical_efficiency,3)}'
      disp = CameraDisplay(camgeom,title=title)
      #disp.highlight_pixels(pixel_mask)
      disp.image = event.r1.tel[1].waveform.sum(axis = 1)
      #disp.image = event.dl1.tel[1].image
      disp.cmap = plt.cm.RdBu_r
      disp.add_colorbar()
      #disp.set_limits_percent(95)
      plt.show()
   #    for pixel in event.r1.tel[1].waveform[pixel_mask]:
   #       #print(pixel)
   #       arr_max_muons_mask.append(np.max(pixel))
   #       arg_max_muons_mask.append(np.argmax(pixel))
         
   #    for pixel in event.r1.tel[1].waveform:
   #       arr_max_muons.append(np.max(pixel))
   #       arg_max_muons.append(np.argmax(pixel))
   # else:
   #    for pixel in event.r1.tel[1].waveform[pixel_mask]:
   #       arr_max_protons_mask.append(np.max(pixel))
   #       arg_max_protons_mask.append(np.argmax(pixel))
   #    for pixel in event.r1.tel[1].waveform:
   #       arr_max_protons.append(np.max(pixel))
   #       arg_max_protons.append(np.argmax(pixel))


   #print(f"number {i} contain event number {event.index.event_id}")


plt.close()

In [ ]:
print(event.mon)


In [ ]:
filename = f'/Users/vdk/RealLST/typical_run_data/LST-1.4.Run14948.0001.fits.fz'
source = EventSource(filename, config = Config(cfg))

event_iterator = iter(source)

arr_max_muons = []
arg_max_muons = []

arr_max_muons_mask = []
arg_max_muons_mask = []

arr_max_protons = []
arg_max_protons = []

arr_max_protons_mask = []
arg_max_protons_mask = []


variance_muon = []
variance_proton = []

muon_n_pixel = []

muon_pixel_number = []
proton_pixel_number = []

for iter_number, event in enumerate(event_iterator):
   #calib(event)
   #image_processor(event)
   #muon_processor(event)
   tmp_var_muon = []
   tmp_var_proton = []
   #print("n pixels = ", event.dl1.tel[1].parameters.morphology.n_pixels)
   #n_pixels.append(event.dl1.tel[1].parameters.morphology.n_pixels)
   #if event.dl1.tel[1].parameters.morphology.n_pixels > 0 and event.dl1.tel[1].parameters.morphology.n_pixels < 1855:
   if True:
      muon_pix_temp = 0
      proton_pix_temp = 0
      pixel_mask = np.array(np.max(event.r1.tel[1].waveform, axis = 1) > 2.5, dtype = bool)
      if iter_number in muon_rings:
         #muon_n_pixel.append(event.dl1.tel[1].parameters.morphology.n_pixels)
         
         for pixel in event.r1.tel[1].waveform[pixel_mask]:
            muon_pix_temp += 1
         #print(pixel)
            #arr_max_muons_mask.append(np.max(pixel))
            arg_max_muons_mask.append(np.argmax(pixel))
         
         # for pixel in event.r1.tel[1].waveform:
         #    #arr_max_muons.append(np.max(pixel))
         #    #arg_max_muons.append(np.argmax(pixel))
         
         for pixel in event.r1.tel[1].waveform[:, 7:12]:
            tmp_var_muon.append(np.var(pixel))
         muon_pixel_number.append(muon_pix_temp)

      else:
         for pixel in event.r1.tel[1].waveform[pixel_mask]:
            proton_pix_temp += 1
            #arr_max_protons_mask.append(np.max(pixel))
            arg_max_protons_mask.append(np.argmax(pixel))
         # for pixel in event.r1.tel[1].waveform:
         #    arr_max_protons.append(np.max(pixel))
         #    arg_max_protons.append(np.argmax(pixel))
         for pixel in event.r1.tel[1].waveform[:, 7:12]:
            tmp_var_proton.append(np.var(pixel))
         proton_pixel_number.append(proton_pix_temp)
   variance_muon.append(tmp_var_muon)
   variance_proton.append(tmp_var_proton)

In [ ]:
from astropy.time import Time


tai_mjd_timestamp = event.trigger.tel[1].time

datetime_obj = Time(tai_mjd_timestamp, format='mjd', scale='tai').to_datetime()

print(datetime_obj)


In [ ]:
tai_mjd_timestamp = event.trigger.tel[1].time

datetime_obj = Time(tai_mjd_timestamp, format='unix_tai', scale='tai').to_datetime()

print(datetime_obj)

In [ ]:
print(event.trigger.tel[1].time.to_datetime())

In [ ]:
pixel_mask = np.array(np.max(event.r1.tel[1].waveform, axis = 1) > 2, dtype = bool)
for pixel in event.r1.tel[1].waveform[pixel_mask]:
    print(pixel.shape)

In [ ]:
plt.hist(muon_pixel_number,range=(0,200), density = True)
plt.hist(proton_pixel_number, range=(0,200), density = True)


In [ ]:
key = 434
camgeom = source.subarray.tel[1].camera.geometry
title=f"event{key}_fit{round(muon_list[key].muon.tel[1].efficiency.optical_efficiency,3)}_width{round(muon_list[key].muon.tel[1].efficiency.width.to_value(),3)}"
disp = CameraDisplay(camgeom,title=title)
#disp.image = event.r1.tel[teln].waveform[:,11]
#disp.image = event.dl0.tel[teln].waveform[:,11]
disp.image = muon_list[key].dl1.tel[1].image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
plt.show()

### Build images + waveforms of events

In [ ]:
#event = next(event_iterator)
# There are 13247 events

source = EventSource(filename, config = Config(cfg))
event_iterator = iter(source)
config = replace_config(standard_config, muon_processor_config)
config = replace_config(config, muon_processor_config_2)
optical_efficiency_list = []
muon_list = {}
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray, config = Config(config))
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray, config = Config(config))
n_pixels = []

for iter_number,event in enumerate(event_iterator):
   #print(iter_number)
   calib(event)
   image_processor(event)
   muon_processor(event)
   #print("n pixels = ", event.dl1.tel[1].parameters.morphology.n_pixels)
   n_pixels.append(event.dl1.tel[1].parameters.morphology.n_pixels)
   if event.dl1.tel[1].parameters.morphology.n_pixels > 50 and event.dl1.tel[1].parameters.morphology.n_pixels < 150:
      j = event
      teln = 1
      fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
      fig.subplots_adjust(hspace=0.2)
      ax = axes[0]
      camgeom = source.subarray.tel[1].camera.geometry
      title=f"event{iter_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
      disp = CameraDisplay(camgeom,title=title, ax = ax)
      #disp.image = event.r1.tel[teln].waveform[:,11]
      #disp.image = event.dl0.tel[teln].waveform[:,11]
      disp.image = j.dl1.tel[teln].image
      disp.cmap = plt.cm.RdBu_r
      disp.add_colorbar()
      disp.set_limits_percent(95)
      bx = axes[1]
      for pix_id in range(0,1855):
         #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
         bx.plot(j.r1.tel[teln].waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
      plt.show()
   if not np.isnan(event.muon.tel[1].efficiency.width):
      muon_list[iter_number] = event
   #print(f"number {i} contain event number {event.index.event_id}")
plt.close()

## Mono sims

In [ ]:
event_waveforms = {}
muon_waveforms = []
#for filename in os.listdir('/Users/vdk/UnrealLST/waveforms/'):
for filename in os.listdir('/Users/vdk/RealLST/waveforms/'):
    #event_waveforms[filename[10:16]] = np.load('/Users/vdk/UnrealLST/waveforms/' + filename)
    event_waveforms[filename[10:16]] = np.load('/Users/vdk/RealLST/waveforms/' + filename)
    #muon_waveforms.append( np.load('/Users/vdk/RealLST/waveforms/' + filename, allow_pickle=True))

# proton_waveforms = []
# for filename2 in os.listdir('/Users/vdk/UnrealLST/waveformsProton/'):
#     proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveformsProton/' + filename2, allow_pickle=True))

#proton_waveforms = []
#for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_flag/'):
#    proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True))

str_rings = [str(number) for number in flagged_rings_sim]
proton_waveforms = []
flag = True
counter = 0
# for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_flag/'):
# #for filename2 in os.listdir('/Users/vdk/UnrealLST/waveformsProton/'):
#     for i in str_rings:
#         if filename2[20:26] in str_rings:
#             flag = False
#             #print(filename2[20:26])
#             break
#     if flag:
#         flag = True
#         #muon_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True))
#         # ADDITIONAL RINGS CONTAMINATED AND NOT FULL:
#         event_waveforms[filename2[20:26]] = np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True)
#         #proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True))
        
#     else:
#         counter += 1
#         #print(filename2[20:26])
#         proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True))
#     if counter == 1000:
#        break

counter = 0
proton_waveforms = []
#for filename2 in os.listdir('/Users/vdk/UnrealLST/waveformsProton/'):
for filename2 in os.listdir('/Users/vdk/RealLST/waveforms_protons/'):
  proton_waveforms.append(np.load('/Users/vdk/RealLST/waveforms_protons/' + filename2, allow_pickle=True))
  if counter == 1000:
    break



min_time = 9
max_time = 15

arr_max_muons = []
arg_max_muons = []

arr_max_muons_mask = []
arg_max_muons_mask_sim = []

variance_muon_sim = []


muon_pixel_number_sim = []

for key in event_waveforms:
   tmp_var_muon = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(event_waveforms[key], axis = 1) > 2, dtype = bool)
      for pixel in event_waveforms[key][pixel_mask]:
        arg_max_muons_mask_sim.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in event_waveforms[key][:, min_time:max_time]:
        tmp_var_muon.append(np.var(pixel))
   
   variance_muon_sim.append(tmp_var_muon)
   muon_pixel_number_sim.append(tmp_pixel)


arg_max_protons = []


arg_max_protons_mask_sim = []

variance_proton_sim = []

proton_pixel_number_sim = []

for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_sim.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_sim.append(tmp_var_proton)
   proton_pixel_number_sim.append(tmp_pixel)


In [ ]:
#arg_max_muons_mask_sim

In [ ]:
str_rings = [str(number) for number in flagged_rings_sim]
proton_waveforms = []
flag = True
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_flag/'):
    for i in str_rings:
        if filename2[20:26] in str_rings:
            flag = False
            print(filename2[20:26])
            break
    if flag:
        flag = True
        muon_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True))
        #proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True))
        
    else:
        #print(filename2[20:26])
        proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_flag/' + filename2, allow_pickle=True))


In [ ]:

# for waveform in muon_waveforms:
#     camgeom = source.subarray.tel[1].camera.geometry
#     disp = CameraDisplay(camgeom,title='1')
#     disp.image = waveform.sum(axis = 1)
#     disp.cmap = plt.cm.RdBu_r
#     disp.add_colorbar()
#     disp.set_limits_percent(95)
#     plt.show()

# plt.close()

#for waveform in muon_waveforms:
for waveform in proton_waveforms:    
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    title =1 
    disp = CameraDisplay(camgeom,title=title, ax = ax)
    disp.image = waveform.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    bx = axes[1]
    for pix_id in range(0,1855):
        bx.plot(waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    plt.show()

plt.close()

In [ ]:
for_test = str(6)
test = ['635903.t', 'fdfsdfs', 'fdsfds']

for i in test:
    if for_test in i:
        print("Wohoo")

### Build R1 distributions 

#### Value of R1 max amplitude 

In [ ]:
# MUONS + PROTONS

k, z, l = plt.hist(arr_max_muons_mask,bins = 100, range = (0,10), density = True, alpha = 0.7, label = 'muons')
k, z, l = plt.hist(arr_max_protons_mask,bins = 100, range = (0,10), density = True, alpha = 0.35, label = 'protons')

plt.legend()
plt.xlabel("R1 waveform max amplitude [p.e. / ns]")
plt.ylabel("Probability density")
plt.grid()
#plt.ylim(0,1.1)

In [ ]:
# MUONS + PROTONS with NSB

k, z, l = plt.hist(arr_max_muons,bins = 100, range = (0,10), density = True, alpha = 0.7, label = 'muons')
k, z, l = plt.hist(arr_max_protons,bins = 100, range = (0,10), density = True, alpha = 0.35, label = 'protons')

plt.legend()
plt.xlabel("R1 waveform max amplitude [p.e. / ns]")
plt.ylabel("Probability density")
plt.grid()
#plt.ylim(0,1.1)

In [ ]:
new_arg = [i+1 for i in arg_max_protons_mask_sim]

#### Time of R1 max amplitude

In [ ]:
plt.figure(figsize = (9,6))
bin_number = 35
# Time distribution
k, z, l = plt.hist(arg_max_protons_mask, bins = bin_number-4, color = '#1f77b4', density = True, alpha = 0.35, label = 'protons in observation')
muons_val, muons_bin, l = plt.hist(arg_max_muons_mask, bins = bin_number-4, density = True,color = 'orange', alpha = 0.35, label = 'muons in observation')
k, z, l = plt.hist(arg_max_protons_mask_sim, bins = bin_number, density = True, color = '#1f77b4', alpha = 1, label = 'protons in simulation', histtype = 'step')
#k, z, l = plt.hist(new_arg, bins = bin_number, density = True, color = '#1f77b4', alpha = 1, label = 'protons in simulation', histtype = 'step')
k, z, l = plt.hist(arg_max_muons_mask_sim, bins = bin_number, density = True, color = 'orange', alpha = 1, label = 'muons in simulation', histtype = 'step')




plt.legend()
plt.xlabel("Time of max R1 amplitude")
plt.ylabel("Probability density")
plt.grid(alpha = 0.5)
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_timesample_R1cut3.png")


In [ ]:
muons_val

#### Variance distribution

In [ ]:
#var_hist_m_short = [np.mean(np.array(item)) for item in variance_muon] # only for considered time span (7:12)
#var_hist_p_short = [np.mean(np.array(item)) for item in variance_proton]

var_hist_m_short_sim = [np.mean(np.array(item)) for item in variance_muon_sim] # only for considered time span (7:12)
var_hist_p_short_sim = [np.mean(np.array(item)) for item in variance_proton_sim]

#outliers = [var for var in var_hist_p_short if var > 0.4]
#len(outliers)/len(var_hist_p_short)



In [ ]:
bin_number = 15
plt.figure(figsize = (9,6))
#z,x,c = plt.hist(var_hist_p_short, alpha = 0.35, density = True,color = '#1f77b4', bins = bin_number, range =(0,1), label='protons in observation')
#plt.hist(var_hist_m_short, alpha = 0.35, density = True, color = 'orange', bins = bin_number, range=(0,1), label='muons in observation')
plt.hist(var_hist_p_short_sim, alpha = 1, density = True, color = '#1f77b4', bins = bin_number, range =(0,1), label='protons in simulation', histtype='step')
plt.hist(var_hist_m_short_sim, alpha = 1, density = True, bins = bin_number, color = 'orange', range=(0,1), label='muons in simulation', histtype='step')
plt.grid(alpha = 0.5)
plt.legend()
plt.xlabel('Mean of variance per pixel for every event')
plt.ylabel('Probability density')
plt.title('Distribution of mean for variance per pixel for timebins [7:12] - obs; [9:14] - sim')
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_variance_R1cut2_7-11timebin.png")

In [ ]:
protons_outlier = [250 for i in proton_pixel_number if i >= 250]
protons_outlier_sim = [250 for i in proton_pixel_number_sim if i >= 250]

final_proton_number = proton_pixel_number + protons_outlier
final_proton_number_sim = proton_pixel_number_sim + protons_outlier_sim

#### Pixel number distribution

In [ ]:
bin_number = 10
plt.figure(figsize = (9,6))
z,x,c = plt.hist(final_proton_number, alpha = 0.35, density = True,color = '#1f77b4', bins = bin_number, range =(0,250), label='protons in observation')
plt.hist(muon_pixel_number, alpha = 0.35, density = True, color = 'orange', bins = bin_number, range=(0,250), label='muons in observation')
plt.hist(final_proton_number_sim, alpha = 1, density = True, color = '#1f77b4', bins = bin_number, range =(0,250), label='protons in simulation', histtype='step')
plt.hist(muon_pixel_number_sim, alpha = 1, density = True, bins = bin_number, color = 'orange', range=(0,250), label='muons in simulation', histtype='step')
plt.grid(alpha = 0.5)
plt.legend()
plt.xlabel('Number of pixels on the image with R1 amplitude > 2.5')
plt.ylabel('Probability density')
plt.title('Distribution of pixel number for simulation and real data (overflow bin 250)')
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_variance_R1cut2_7-11timebin.png")

## LSTCHAIN processing

### Real Data on the cluster

In [ ]:
muon_file = '/Users/vdk/RealLST/output/muons_LST-1.Run14948.0001.fits' # My Local output
#muon_file = '/Users/vdk/RealLST/muons_LST-1.Run14948.0001._DATACHECK.fits' # Official Cluster output
#muon_file = '/Users/vdk/muons_LST-1.Run14948.0001.fits' # My Cluster output
muon_file = '/Users/vdk/RealLST/muons_LST-1.Run14948.0001.fits' # My NEW Cluster output WORKING!
muon_file = '/Users/vdk/RealLST/output_14985/muons_LST-1.Run14985.0001.fits' # My NEW Cluster output WORKING!

#muon_file = '/Users/vdk/RealLST/typical_run_data/muons_LST-1.Run14948.0001.fits'
#muon_file = '/Users/vdk/RealLST/dir_flag/muons_LST-1.Run14948.0001.fits'
#muon_file = '/Users/vdk/RealLST/cleaning/muons_LST-1.Run14948.0001_noclean.fits'
#muon_file = '/Users/vdk/RealLST/cleaning/muons_LST-1.Run14948.0001_clean15-36.fits'
#muon_file = '/Users/vdk/RealLST/MuonFitsForCompare/muons_LST-1.Run14948.0001_minPix90.fits' # min pix cut 50 r1 = 2.5
#muon_file = '/Users/vdk/RealLST/cleaning/muons_LST-1.Run14948.0001_15.fits' # CLEANING from cluster for full 1 run


#dl1_file = '/Users/vdk/RealLST/output/cta_env_output/dl1_LST-1.Run14948.0001.h5'
dl1_file = '/Users/vdk/RealLST/dl1_LST-1.Run14948.0001.h5'
dl1_file = '/Users/vdk/RealLST/output_14985/dl1_LST-1.Run14985.0001.h5'
#dl1_file = '/Users/vdk/dl1_LST-1.Run14948.0001.h5'



#### Muons from fits file

In [ ]:
hdul = fits.open(muon_file)
hdul.info()
opt_eff = []
containment = []
completeness = []
n_ring_pixels = []
event_id = []
efficiency_dict = {}
# element number 10 is muon efficiency, so in such way we can check it
for i in range(len(hdul[1].data)):
    #print(hdul[1].data[i])
    opt_eff.append(hdul[1].data[i][10])
    containment.append(hdul[1].data[i][11])
    completeness.append(hdul[1].data[i][12])
    n_ring_pixels.append(hdul[1].data[i][-4])
    event_id.append(hdul[1].data[i][0])
    efficiency_dict[hdul[1].data[i][0]] = hdul[1].data[i][10]


In [ ]:
print("Optical Efficiency value = ", round(np.mean(opt_eff),3))
print("Optical Efficiency relative std value = ", round(np.std(opt_eff)/len(opt_eff)**0.5,3))
print("Number of fitted muons = ", len(opt_eff))
print("Event IDs", event_id)


In [ ]:
#cleaned_efficiency_dict = efficiency_dict.copy()

#print(efficiency_dict.keys())
#print(cleaned_efficiency_dict.keys())

print(opt_eff)
plt.scatter([i for i in range(len(opt_eff))], opt_eff)

In [ ]:
print("Optical Efficiency value = ", round(np.mean(opt_eff),3))
print("Optical Efficiency relative std value = ", round(np.std(opt_eff)/len(opt_eff)**0.5,3))
print("Number of fitted muons = ", len(opt_eff))


In [ ]:
print(f"completeness = {sorted(completeness)}")
print(f"containment = {sorted(containment)}")
print(f"n_ring_pixels = {sorted(n_ring_pixels)}")

In [ ]:
plt.figure(figsize=(7,5))
plt.hist(completeness, alpha = 0.3, label = 'completeness', bins = 10, density = True)
plt.hist(containment, alpha = 0.3, label = 'containment', bins = 10, density = True)
plt.grid()
plt.xlim(0,1)
plt.legend()

### Graphs

In [ ]:
mean_range = [i-1 for i in range(len(opt_eff)+2)]
y_mid = [np.mean(opt_eff) for i in mean_range]
#y_mid = [np.mean(nice_opt_eff[:i+1]) for i in range(len(nice_opt_eff[:i+1]))]

y_upper = [np.mean(opt_eff)+np.std(opt_eff) for i in mean_range]
y_lower = [np.mean(opt_eff)-np.std(opt_eff) for i in mean_range]

x_range = [i for i in range(len(opt_eff))]




#std_arr = [np.std(optical_coef)/np.mean(optical_coef) for i,j in enumerate(optical_coef)]
std_arr = [np.std(opt_eff)/i**0.5 for i,j in enumerate(opt_eff)]
#std_arr_big = [np.std(opt_eff) for i,j in enumerate(opt_eff)]

#std_arr_3 = [np.std(new_plot)/i**0.5 for i,j in enumerate(new_plot)]
#std_arr_3_big = [np.std(new_plot) for i,j in enumerate(new_plot)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


plt.figure(figsize = (9,6))
plt.ylim(0.05,0.3)
#plt.ylim(0.095,0.265)
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency value")
#plt.title("Opt eff for muon rings after time sample cleaning, pix > 50, intensity<0.2")
plt.title("Optical efficiency for events in the subrun 14948.0001")
#plt.title("Events after island cleaning (size $\leq 4$)")


plt.grid(alpha=0.5)
#plt.ylim(0.05,0.3)
#plt.scatter(new_ind, new_plot, color = 'orange')


#y_upper = [j+std_arr[-1] for i,j in enumerate(y_mid)]
#y_lower = [j-std_arr[-1] for i,j in enumerate(y_mid)]


y_upper = [j+std_arr[-1] for i,j in enumerate([np.mean(opt_eff) for i in x_range])]
y_lower = [j-std_arr[-1] for i,j in enumerate([np.mean(opt_eff) for i in x_range])]

plt.errorbar(x_range, opt_eff, yerr=std_arr, fmt='o', ms = 5, alpha = 0.8, label="")#, c = 'blue')#, s = 100)#, c= 'black')
#plt.errorbar(new_ind, new_plot, yerr=std_arr_3, fmt='o', ms = 5, alpha = 0.8, label="original") #, c = 'blue')#, s = 100)#, c= 'black')
#plt.errorbar(new_ind, new_plot, yerr=new_std, fmt='o', ms = 5, alpha = 0.8, label="original") #, c = 'blue')#, s = 100)#, c= 'black')

#plt.scatter(new_ind, new_plot, alpha = 0.9, color = "orange", label="original") #, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(mean_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
plt.fill_between(x_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
#plt.plot(mean_range, y_mid, label = "mean", lw = 1.5, color='green')

#plt.plot(x_range, y_mid, label = "mean", lw = 1.5, color='green')
plt.plot(x_range, [np.mean(opt_eff) for i in x_range], label = "mean", lw = 1.5, color='green')

#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'r', alpha = 0.3, label = 'mean +/- $1\sigma$')

plt.xlim(-1,len(opt_eff))
plt.legend()
#plt.savefig("/Users/vdk/ImagePurgatorium/CleanedOptEff_pix>50_int=01_16timebin_complet>05_RelStdErr_fdsfs.png")
#plt.savefig("/Users/vdk/ImagePurgatorium/LST1_14948_OptEffRaw.png", dpi=900)

#### DL1 data level

In [ ]:
dl1_file = '/Users/vdk/dl1_simtel_corsika_run322.h5'
fi = h5py.File(dl1_file, 'r')
keys = list(fi.keys())

def printname(name):
    print(name)

# Show all included tables
fi.visit(printname)

In [ ]:
length = 0
for item in fi['dl1/event/telescope/parameters/LST_LSTCam'][:10]:
    for i in item:
        print(i)
    #length+=1

In [ ]:
import pandas as pd
from astropy.table import Table
from ctapipe.instrument import SubarrayDescription
from lstchain.io.io import dl1_params_lstcam_key, dl1_images_lstcam_key
from lstchain.io import read_calibration_file

filename = '/Users/vdk/dl1_simtel_corsika_run32.h5'
subarray = SubarrayDescription.from_hdf(filename)

images = Table.read(filename, path=dl1_images_lstcam_key)['image']
parameters = pd.read_hdf(filename, key=dl1_params_lstcam_key)

In [ ]:
pd.set_option('display.max_columns', None)
for i in parameters['tel_id'][:]:
    if i != 1:
        print("hehe")

### Real data on the local machine

### Simulation

In [ ]:
#filename = '/Users/vdk/UnrealLST/muons_new_sim/muon_table_run1.fits'
#filename = '/Users/vdk/UnrealLST/muon_table.fits'
filename = '/Users/vdk/UnrealLST/optical_efficiency_scan/muon_table_run249.fits'
hdul = fits.open(filename)
hdul.info()
opt_eff = []
containment = []
completeness = []
n_ring_pixels = []
event_id = []
# element number 10 is muon efficiency, so in such way we can check it
for i in range(len(hdul[1].data)):
    print(hdul[1].data[i])
    opt_eff.append(hdul[1].data[i][10])
    containment.append(hdul[1].data[i][11])
    completeness.append(hdul[1].data[i][12])
    n_ring_pixels.append(hdul[1].data[i][-4])
    event_id.append(hdul[1].data[i][0])

opt_eff = np.array(opt_eff)
opt_eff = opt_eff[np.logical_not(np.isnan(opt_eff))]

In [ ]:
print(np.mean(opt_eff))
print(len(opt_eff))

In [ ]:
opt_eff_2 = []
containment = []
completeness = []
n_ring_pixels = []
event_id = []
muon_id = []
mc_energy = []
opt_eff_multiplier = '0634'
for i in range(1,500):
    #filename = f'/Users/vdk/UnrealLST/muons_new_sim/muon_table_run{i}.fits'
    #filename = f'/Users/vdk/UnrealLST/optical_efficiency_scan/table_muons_{opt_eff_multiplier}/muon_table_run{i}.fits'
    #filename = f'/Users/vdk/UnrealLST/table_muon_lz/muon_table_run{i}.fits' 
    filename = f'/Users/vdk/UnrealLST/table_muons_low_photons/muon_table_run{i}.fits' 
    #print(filename)
    hdul = fits.open(filename)
    # element number 10 is muon efficiency, so in such way we can check it
    for i in range(len(hdul[1].data)):
        #print(hdul[1].data[i])
        #print(hdul[1].data[i][10])
        #print(hdul[1].data[i])
        #containment.append(hdul[1].data[i][11])
        #completeness.append(hdul[1].data[i][12])
        #n_ring_pixels.append(hdul[1].data[i][-4])
        if not np.isnan(hdul[1].data[i][10]):
            event_id.append(hdul[1].data[i][0])

            #print(hdul[1].data[i][10])
            opt_eff_2.append(hdul[1].data[i][10])
            muon_id.append(hdul[1].data[i][0])
            mc_energy.append(hdul[1].data[i][2])

opt_eff_2 = np.array(opt_eff_2)
#opt_eff_2 = opt_eff_2[np.logical_not(np.isnan(opt_eff_2))]

In [ ]:
print("Mean optical efficiency = ", np.mean(opt_eff_2))
print("Std optical efficiency = ", np.std(opt_eff_2))
print("Number of fitted muons = ", len(opt_eff_2))
print(f"muons_{opt_eff_multiplier} = ", event_id)
#print(muon_id)
#print(len(muon_id))
#print(opt_eff_2)
#print(np.std(opt_eff_2))
print( np.mean(opt_eff_2) + np.std(opt_eff_2))
print(mc_energy)


In [ ]:
from pyirf import spectral
simulated_spectrum = spectral.PowerLaw(normalization = 1 * spectral.DIFFUSE_FLUX_UNIT, index = -2, e_ref =1* u.TeV)
target_spectrum = spectral.PowerLaw(normalization = 1 * spectral.DIFFUSE_FLUX_UNIT, index = -2.7, e_ref =1* u.TeV)
pyirf_weighted = [spectral.calculate_event_weights(energy*u.TeV, target_spectrum=target_spectrum, simulated_spectrum=simulated_spectrum) for energy in mc_energy]
sum(pyirf_weighted)

In [ ]:
file = open('/Users/vdk/low_zenith_sim_muons.pkl', 'wb')

# dump information to that file
pickle.dump(event_id, file)

# close the file
file.close()

In [ ]:
file = open('/Users/vdk/low_zenith_sim_muons.pkl', 'rb')

# dump information to that file
data = pickle.load(file)

In [ ]:
800*1000

In [ ]:
print(29/17000)
print(9/7300)
0.78+0.0784

In [ ]:
# run = 6
# length = 0
# for run in range(1,4):
#     dl1_file = f'/Users/vdk/UnrealLST/optical_efficiency_scan/dl1_files/dl1_corsika_theta_10.0_az_248.117_run{run}.h5'
#     #dl1_file = f'/Users/vdk/UnrealLST/dl1_files/dl1_simtel_corsika_theta_30.390_az_93.640_run{run}.h5'
#     fi = h5py.File(dl1_file, 'r')
#     keys = list(fi.keys())
#     for item in fi['dl1/event/telescope/parameters/LST_LSTCam']:
#     #print(item)
#         length+=1

length = 0
for run in range(5,500):
    #dl1_file = f'/Users/vdk/UnrealLST/optical_efficiency_scan/dl1_files/dl1_corsika_theta_10.0_az_248.117_run{run}.h5'
    dl1_file = f'/Users/vdk/opt_eff_0784/dl1_corsika_theta_10.0_az_248.117_run{run}.h5'
    #dl1_file = f'/Users/vdk/UnrealLST/dl1_files/dl1_simtel_corsika_theta_30.390_az_93.640_run{run}.h5'
    fi = h5py.File(dl1_file, 'r')
    keys = list(fi.keys())
    for item in fi['dl1/event/telescope/parameters/LST_LSTCam']:
    #print(item)
        length+=1

def printname(name):
    print(name)

#fi.visit(printname)
length

In [ ]:
dl1_file = f'/Users/vdk/opt_eff_0784/dl1_corsika_theta_10.0_az_248.117_run1.h5'
#dl1_file = f'/Users/vdk/UnrealLST/dl1_files/dl1_simtel_corsika_theta_30.390_az_93.640_run{run}.h5'
fi = h5py.File(dl1_file, 'r')
keys = list(fi.keys())
#for item in fi['dl1/event/telescope/parameters/LST_LSTCam']:
    
fi.visit(printname)

In [ ]:
for item in fi['dl1/event/subarray/trigger']:
    if item[3] != 1:
        print(item[3] == 1)

In [ ]:
for item in fi['dl1/event/telescope/image/LST_LSTCam'][0]:
    print(item.shape)

fi.visit(printname)

In [ ]:
#indices.index(4431008)
indices.index(5405008)

In [ ]:
#eventik = muon_list[3270]
#print("number of pixels = ", eventik.dl1.tel[1].parameters.morphology.n_pixels)
#pixel_mask = np.array(eventik.dl1.tel[1].image > 2, dtype = bool)
plt.figure(figsize = (15,9))
camgeom = source.subarray.tel[1].camera.geometry
title = f'1'
disp = CameraDisplay(camgeom,title=title)
disp.highlight_pixels(pixel_mask)
#disp.image = fi['dl1/event/telescope/image/LST_LSTCam'][413][2]
disp.image = fi['dl1/event/telescope/image/LST_LSTCam'][0][4]
#disp.image = eventik.r1.tel[1].waveform.sum(axis=1)
disp.cmap = plt.cm.Reds
disp.add_colorbar()
#disp.set_limits_percent(95)
plt.show()

## Calibpipe processing

### Real Data

In [ ]:
# Треба завантажити конфіг в івент соурс
filename = f'/Users/vdk/RealLST/typical_run_data/LST-1.1.Run14948.0001.fits.fz'
source = EventSource(filename)
int_cut = 0.1
min_pixel = 50

config_filename = '/Users/vdk/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'

with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" :{
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/RealLST/data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/RealLST/data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/RealLST/data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/RealLST/data/time_calibration.Run08349.0000.h5'
      }})

config = replace_config(standard_config, muon_processor_config)
#onfig = replace_config(config, muon_processor_config_2)
config = replace_config(config, custom_config)

image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray, config = Config(config))
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray, config = Config(config))


In [ ]:
source = EventSource(filename, config = Config(cfg))

event_iterator = iter(source)
optical_eff = []
muon_waveforms = []

for iter_number, event in enumerate(event_iterator):
   calib(event)
   image_processor(event)
   muon_processor(event)
   if not np.isnan(event.muon.tel[1].efficiency.optical_efficiency):
      optical_eff.append(event.muon.tel[1].efficiency.optical_efficiency)
      muon_waveforms.append(event.r1.tel[1].waveform)


In [ ]:
print("number of fitted muon rings = ", len(optical_eff))
print("optical efficiency = ", np.mean(optical_eff))

In [ ]:
for i,waveform in enumerate(muon_waveforms):
    plt.figure(figsize = (18,12))
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'{i}'
    disp = CameraDisplay(camgeom,title=title)
    #disp.highlight_pixels(pixel_mask)
    disp.image = waveform.sum(axis = 1)
    disp.cmap = plt.cm.Reds
    disp.add_colorbar()
    disp.set_limits_percent(95)
    #disp.set_limits_minmax(0,20)
    plt.show()

In [ ]:
ctapipe.__version__

### Simulated Data

In [ ]:
for i, event in enumerate(event_iterator):
    print(event.index.event_id)

In [ ]:
# in 1st run we have two rings 5405008 and 4431008
filename = '/Users/vdk/simtel_corsika_theta_30.390_az_93.640_run1.simtel.gz'
source = EventSource(filename)
event_iterator = iter(source)
for i, event in enumerate(event_iterator):
    #if 1 == event.trigger.tels_with_trigger[0]:
    if event.index.event_id == 5405008:
      fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
      fig.subplots_adjust(hspace=0.2)
      ax = axes[0]
      camgeom = source.subarray.tel[1].camera.geometry
      #title=f"event{iter_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
      disp = CameraDisplay(camgeom,title=title, ax = ax)
      #disp.image = event.r1.tel[teln].waveform[:,11]
      #disp.image = event.dl0.tel[teln].waveform[:,11]
      disp.image = event.r1.tel[1].waveform.sum(axis = 1)
      disp.cmap = plt.cm.RdBu_r
      disp.add_colorbar()
      disp.set_limits_percent(95)
      bx = axes[1]
      for pix_id in range(0,1855):
         #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
         bx.plot(event.r1.tel[1].waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
      plt.show()
    #if i == 20:
    #   break
plt.close()        

In [ ]:
# TIME CLEANING ALGO + ISLANDS CLEANING

int_cut = 0.3
min_pixel = 160

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

counter = 0
cleaned_event_container = {}
r1_cut = 2
min_pixel_cut = 60
max_pixel_cut = 200
low_threshold_variance = 0.1
up_threshold_variance = 0.4
max_time = 14

image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)

for run in range(460,481):
    filename = f'/Users/vdk/simulations/corsika_theta_30.390_az_93.640_run{run}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):
        counter += 1
        calib(event)
        image_processor(event)
        muon_processor(event)   
        if not np.isnan(event.muon.tel[1].efficiency.width):          
            opt_eff_calib.append(event.muon.tel[1].efficiency.optical_efficiency)   






In [ ]:
filename = '/Users/vdk/RealLST/data/LST-1.4.Run14948.0001.fits.fz'
source = EventSource(filename)
event_iterator = iter(source)
for i,event in enumerate(event_iterator):
    counter += 1
    calib(event)
    image_processor(event)
    muon_processor(event)   
    if not np.isnan(event.muon.tel[1].efficiency.width):          
        opt_eff_calib.append(event.muon.tel[1].efficiency.optical_efficiency)  

## Graph Building

In [ ]:
M_n = 179
P_n = 53000-M_n
cuts = np.array([30, 40, 50, 60, 70, 80]) #, 90])
muons_detected = np.array([179, 179, 61, 38, 19, 10]) #, 4])
protons_detected = np.array([519, 276, 141, 79, 34, 18]) #, 9]) - muons_detected

true_negative = P_n - protons_detected
false_negative = M_n - muons_detected 
R1_cut = 2.5

plt.rcParams.update({'font.size': 12, 'figure.figsize': (9,9)})
fig, axs = plt.subplots(2, 2)
fig.subplots_adjust(hspace=0.2, wspace = 0.35)
fig.suptitle(f"Muon flagging based on choosing N pixels which have R1 waveforms values > {R1_cut}, timesample[7:12], 0.1 < mean(variance) < 0.4", fontsize = 15)

axs[0, 0].plot(cuts, muons_detected/M_n)
axs[0, 0].set_title('True Positive')
#axs[0, 0].set_xlabel("Number of cutted pixels", fontsize = 7)
axs[0, 0].set_ylabel("True positive / Mn")

axs[0, 1].plot(cuts, (protons_detected)/M_n, 'tab:orange')
axs[0, 1].set_title('False Positive')
#axs[0, 1].set_xlabel("Number of cutted pixels", fontsize = 7)
axs[0, 1].set_ylabel("False positive / Mn")

axs[1, 0].plot(cuts, true_negative/P_n, 'tab:green')
axs[1, 0].set_title('True Negative')
axs[1, 0].set_xlabel("Number of cutted pixels", fontsize = 10)
axs[1, 0].set_ylabel("True negative / Pn")

axs[1, 1].plot(cuts, false_negative/P_n, 'tab:red')
axs[1, 1].set_title('False Negative')
axs[1, 1].set_xlabel("Number of cutted pixels", fontsize = 10)
axs[1, 1].set_ylabel("False negative / Pn")
for ax in axs.flat:
    ax.grid(alpha = 0.5)

#plt.savefig(f"/Users/vdk/ImagePurgatorium/PixelCut_R1waveform>{R1_cut}_Time+Variance.png")

In [ ]:
# PRECISION RECALL
TP = muons_detected
FP = protons_detected
TN = P_n - protons_detected
FN = M_n - muons_detected

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("precision = ", precision)
print("recal = ", recall)
#fig = plt.figure(figsize = (10,9))
_, ax = plt.subplots(2,1)

ax[0].plot(cuts, precision)
ax[0].grid()
ax[0].set_ylabel("precision")
ax[1].set_xlabel("cuts (min number of pixels)")
ax[1].plot(cuts, recall)
ax[1].set_ylabel("recall")
ax[1].grid()

In [ ]:
# PRECISION RECALL
TP = muons_detected
FP = protons_detected
TN = P_n - protons_detected
FN = M_n - muons_detected

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("precision = ", precision)
print("recal = ", recall)

plt.plot(recall, precision)
plt.grid()
plt.xlabel("Recall")
plt.ylabel("Precision")

#### Precision/recall for muons fitted by lst chain

In [ ]:
M_n = 68
P_n = 53000-M_n
cuts = np.array([30, 40, 50, 60, 70, 80]) #, 90])
muons_detected = np.array([68, 68, 61, 38, 19, 10]) #, 4])
protons_detected = np.array([519, 276, 141, 79, 34, 18]) - muons_detected #, 9]) - muons_detected

M_n_sim = 85
P_n_sim = 14517-85
cuts_sim = np.array([30, 40, 50, 60, 70, 80]) # , 90])
#muons_detected_sim = np.array([85, 83, 79, 67, 55, 49]) #, 45])
muons_detected_sim = np.array([84, 83, 83, 77, 65, 59])
protons_detected_sim = np.array([684, 447, 266, 166, 97, 71])- muons_detected_sim #, 49]) - muons_detected_sim

M_n_sim_mono = 147
#M_n_sim_mono = 51
P_n_sim_mono = 42750-M_n_sim_mono
cuts_sim_mono = np.array([30, 40, 50, 60, 70, 80])
#muons_detected_sim_mono = np.array([51, 51, 50, 44, 36, 27])
#muons_detected_sim_mono = np.array([53, 53, 49, 43, 37, 31])
muons_detected_sim_mono = np.array([144, 141, 121, 90, 60, 44])

#muons_detected_sim_mono = np.array([144, 130, 114, 71, 40, 23]) # 2.5

#protons_detected_sim_mono = np.array([1152, 734, 433, 254, 160, 104])- muons_detected_sim_mono
protons_detected_sim_mono = np.array([1031, 640, 365, 203, 125, 80])- muons_detected_sim_mono

#protons_detected_sim_mono = np.array([920, 508, 278, 161, 83, 51])- muons_detected_sim_mono # 2.5


M_n_full = 8771
P_n_full = 7467508-M_n_full
cuts = np.array([30, 40, 50, 60, 70, 80]) #, 90])
muons_detected_full = np.array([8695, 8686, 8634, 7130, 4422, 3000]) #, 4])
protons_detected_full = np.array([185934, 108332, 65925, 41309, 26309, 15000]) - muons_detected_full #, 9]) - muons_detected
# PRECISION RECALL
TP_full = muons_detected_full
FP_full = protons_detected_full
TN_full = P_n_full - protons_detected_full
FN_full = M_n_full - muons_detected_full
precision_full = TP_full/(TP_full+FP_full)
recall_full = TP_full/(TP_full+FN_full)



# PRECISION RECALL
TP = muons_detected
FP = protons_detected
TN = P_n - protons_detected
FN = M_n - muons_detected
precision = TP/(TP+FP)
recall = TP/(TP+FN)


TP_sim = muons_detected_sim
FP_sim = protons_detected_sim
TN_sim = P_n_sim - protons_detected_sim
FN_sim = M_n_sim - muons_detected_sim
precision_sim = TP_sim/(TP_sim+FP_sim)
recall_sim = TP_sim/(TP_sim+FN_sim)


TP_sim_mono = muons_detected_sim_mono
FP_sim_mono = protons_detected_sim_mono
TN_sim_mono = P_n_sim_mono - protons_detected_sim_mono
FN_sim_mono = M_n_sim_mono - muons_detected_sim_mono
precision_sim_mono = TP_sim_mono/(TP_sim_mono+FP_sim_mono)
recall_sim_mono = TP_sim_mono/(TP_sim_mono+FN_sim_mono)

print("precision = ", precision)
print("recal = ", recall)
#fig = plt.figure(figsize = (10,9))
_, ax = plt.subplots(2,1)

ax[0].plot(cuts, precision, label = 'Observational Data')
ax[0].plot(cuts, precision_sim, label = 'Stereo Simulations')
ax[0].plot(cuts_sim_mono, precision_sim_mono, label = 'Mono Simulations')
#ax[0].plot(cuts, precision_full, label = 'Full Obs')
ax[0].grid()
ax[0].legend()
ax[0].set_ylabel("precision")
ax[1].set_xlabel("cuts (min number of pixels)")
ax[1].plot(cuts, recall, label = 'Observational Data')
ax[1].plot(cuts, recall_sim, label = 'Stereo Simulations')
ax[1].plot(cuts_sim_mono, recall_sim_mono, label = 'Mono Simulations')
#ax[1].plot(cuts, recall_full, label = 'Full Obs')
ax[1].set_ylabel("recall")
ax[1].grid()
ax[1].legend()

In [ ]:
# PRECISION RECALL
TP = muons_detected
FP = protons_detected
TN = P_n - protons_detected
FN = M_n - muons_detected

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("precision = ", precision)
print("recal = ", recall)

#plt.plot(recall, precision)
#plt.plot(recall_sim, precision_sim)
plt.plot(precision, recall)
plt.plot(precision_sim, recall_sim)

plt.grid()
plt.ylabel("Recall")
plt.xlabel("Precision")

In [ ]:
cuts = [30,40,50,60,70,80,90]
true_opt_eff = [0.1654 for i in cuts]
y_upper = [0.1654+(0.002)/(1**0.5) for i in cuts]
y_lower = [0.1654-(0.002)/(1**0.5) for i in cuts]

plt.figure(figsize=(9,6))
flagged_opt_eff = [0.1652, 0.1652, 0.1655, 0.1637, 0.1644, 0.1672, 0.1732]
#opt_eff_err = [np.std(flagged_opt_eff[i])/muons_detected[i]**0.5 for i in range(len(muons_detected))]
opt_eff_err = [0.002, 0.002, 0.002, 0.002, 0.002, 0.003, 0.005]
plt.errorbar(x = cuts, y = flagged_opt_eff, yerr = opt_eff_err,fmt='o', ms = 7, alpha = 1, label = 'optical efficiency after tagging', color = 'black')
plt.fill_between([30,40,50,60,70,80,90], y_upper, y_lower, alpha = 0.3, color = 'green', label = 'optical efficiency without tagging')
plt.plot(cuts, true_opt_eff, color = 'green')#, label = 'without flagging')
plt.xlim(28,92)
plt.ylim(0.15,0.2)
plt.yticks((0.14,0.15,0.16, 0.17, 0.18, 0.19))
plt.grid()
plt.xlabel("cuts on min pixels")
plt.ylabel("optical efficiency")
plt.title("Performance of tagging in terms of ability to correctly reproduce optical efficiency")
plt.legend()

In [ ]:
M_n = 177
P_n = 53000-M_n
cuts = np.array([30, 40, 50, 60, 70, 80]) #, 90])
muons_detected = np.array([176, 176, 116, 69, 29, 15]) #, 4])
protons_detected = np.array([519, 276, 141, 79, 34, 18]) - muons_detected #, 9]) - muons_detected

true_negative = P_n - protons_detected
false_negative = M_n - muons_detected 
R1_cut = 2.5

plt.rcParams.update({'font.size': 12, 'figure.figsize': (9,9)})
fig, axs = plt.subplots(2, 2)
fig.subplots_adjust(hspace=0.2, wspace = 0.35)
fig.suptitle(f"Muon flagging based on choosing N pixels which have R1 waveforms values > {R1_cut}, timesample[7:12], 0.1 < mean(variance) < 0.4", fontsize = 15)

axs[0, 0].plot(cuts, muons_detected/M_n)
axs[0, 0].set_title('True Positive')
#axs[0, 0].set_xlabel("Number of cutted pixels", fontsize = 7)
axs[0, 0].set_ylabel("True positive / Mn")

axs[0, 1].plot(cuts, (protons_detected)/M_n, 'tab:orange')
axs[0, 1].set_title('False Positive')
#axs[0, 1].set_xlabel("Number of cutted pixels", fontsize = 7)
axs[0, 1].set_ylabel("False positive / Mn")

axs[1, 0].plot(cuts, true_negative/P_n, 'tab:green')
axs[1, 0].set_title('True Negative')
axs[1, 0].set_xlabel("Number of cutted pixels", fontsize = 10)
axs[1, 0].set_ylabel("True negative / Pn")

axs[1, 1].plot(cuts, false_negative/P_n, 'tab:red')
axs[1, 1].set_title('False Negative')
axs[1, 1].set_xlabel("Number of cutted pixels", fontsize = 10)
axs[1, 1].set_ylabel("False negative / Pn")
for ax in axs.flat:
    ax.grid(alpha = 0.5)

#plt.savefig(f"/Users/vdk/ImagePurgatorium/PixelCut_R1waveform>{R1_cut}_Time+Variance.png")

#### Precision/recall for all muons in sample

In [ ]:
M_n = 177
P_n = 53000-M_n
cuts = np.array([30, 40, 50, 60, 70, 80]) #, 90])
muons_detected = np.array([176, 176, 116, 69, 29, 15]) #, 4])
protons_detected = np.array([519, 276, 141, 79, 34, 18]) - muons_detected #, 9]) - muons_detected


M_n_sim = 85
P_n_sim = 14517-85
cuts_sim = np.array([30, 40, 50, 60, 70, 80]) # , 90])
muons_detected_sim = np.array([85, 83, 79, 67, 55, 49]) #, 45])
protons_detected_sim = np.array([784, 447, 266, 166, 97, 71])- muons_detected_sim #, 49]) - muons_detected_sim
muons_detected_sim = np.array([83, 83, 83, 77, 65, 59])
protons_detected_sim = np.array([657, 447, 266, 166, 97, 71]) - muons_detected_sim

# PRECISION RECALL
TP = muons_detected
FP = protons_detected
TN = P_n - protons_detected
FN = M_n - muons_detected
precision = TP/(TP+FP)
recall = TP/(TP+FN)


TP_sim = muons_detected_sim
FP_sim = protons_detected_sim
TN_sim = P_n_sim - protons_detected_sim
FN_sim = M_n_sim - muons_detected_sim
precision_sim = TP_sim/(TP_sim+FP_sim)
recall_sim = TP_sim/(TP_sim+FN_sim)

print("precision = ", precision)
print("recal = ", recall)
#fig = plt.figure(figsize = (10,9))
_, ax = plt.subplots(2,1)

ax[0].plot(cuts, precision, label = 'Observational Data')
#ax[0].plot(cuts, precision_sim, label = 'Simulations')
ax[0].grid()
ax[0].legend()
ax[0].set_ylabel("precision")
ax[1].set_xlabel("cuts (min number of pixels)")
ax[1].plot(cuts, recall, label = 'Observational Data')
#ax[1].plot(cuts, recall_sim, label = 'Stereo simulations')
ax[1].set_ylabel("recall")
ax[1].grid()
ax[1].legend()

In [ ]:
# PRECISION RECALL
TP = muons_detected
FP = protons_detected
TN = P_n - protons_detected
FN = M_n - muons_detected

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("precision = ", precision)
print("recal = ", recall)

#plt.plot(recall, precision)
#plt.plot(recall_sim, precision_sim)
plt.plot(precision, recall, label = 'data')
plt.plot(precision_sim, recall_sim, label = 'sim')

plt.grid()
plt.legend()
plt.xlabel("Precision")
plt.ylabel("Recall")

In [ ]:
print(event.trigger.tel[1])

In [ ]:
print(event.r1.tel[1].waveform.shape)

In [ ]:
plt.hist(np.argmax(event.r1.tel[1].waveform, axis = 0), bins = 50)

In [ ]:
plt.hist(np.argmax(event.r0.tel[1].waveform[0], axis = 1))

In [ ]:
muon_rings = [434, 663, 949, 1295, 1355, 1565, 1777, 1933, 3067, 3179, 3270, 3314,3346, 3692, 4076,  5016, 6853, 7723, 7919, 8122, 8605, 9047, 9157, 9239, 9774,  9910, 10486, 10491, 11600, 11781, 12038, 12506, 12969, 13004, 13101] 
len(muon_rings)

In [ ]:
35/6

In [ ]:
image = event.dl1.tel[1].image


In [ ]:
len(image[image > 4])

In [ ]:
plt.figure(figsize = (18,12))
camgeom = source.subarray.tel[1].camera.geometry
title = f'1'
disp = CameraDisplay(camgeom,title=title)
disp.highlight_pixels(pixel_mask)
#disp.image = eventik.dl1.tel[1].image
disp.image = image
disp.cmap = plt.cm.Reds
disp.add_colorbar()
#disp.set_limits_percent(95)
plt.show()

In [ ]:
event.r1.tel[1].waveform.shape[1]  

In [ ]:
calibration_mon = source.r0_r1_calibrator.mon_data.tel[1].calibration

In [ ]:
telescope_id = 1
numsamples = event.r1.tel[telescope_id].waveform.shape[1]  # not necessarily the same as in r0!
bad_pixels_hg = calibration_mon.unusable_pixels[0]
bad_pixels_lg = calibration_mon.unusable_pixels[1]


# Now set to 0 all samples in unreliable pixels. Important for global peak
# integrator in case of crazy pixels!  TBD: can this be done in a simpler
# way?
bad_pixels = bad_pixels_hg | bad_pixels_lg
bad_waveform = np.transpose(np.array(numsamples*[bad_pixels]))

In [ ]:
M_n = 68
P_n = 53000-M_n
cuts = np.array([30, 40, 50, 60, 70, 80]) #, 90])
muons_detected = np.array([68, 68, 61, 38, 19, 10]) #, 4])
protons_detected = np.array([519, 276, 141, 79, 34, 18]) - muons_detected #, 9]) - muons_detected

M_n_sim = 85
P_n_sim = 14517-85
cuts_sim = np.array([30, 40, 50, 60, 70, 80]) # , 90])
#muons_detected_sim = np.array([85, 83, 79, 67, 55, 49]) #, 45])
muons_detected_sim = np.array([84, 83, 83, 77, 65, 59])
protons_detected_sim = np.array([684, 447, 266, 166, 97, 71])- muons_detected_sim #, 49]) - muons_detected_sim

M_n_sim_mono = 147
#M_n_sim_mono = 51
P_n_sim_mono = 42750-M_n_sim_mono
cuts_sim_mono = np.array([30, 40, 50, 60, 70, 80])
muons_detected_sim_mono = np.array([51, 51, 50, 44, 36, 27])
#muons_detected_sim_mono = np.array([53, 53, 49, 43, 37, 31])
muons_detected_sim_mono = np.array([144, 141, 121, 90, 60, 44])

#muons_detected_sim_mono = np.array([144, 130, 114, 71, 40, 23]) # 2.5

#protons_detected_sim_mono = np.array([1152, 734, 433, 254, 160, 104])- muons_detected_sim_mono
protons_detected_sim_mono = np.array([1031, 640, 365, 203, 125, 80])- muons_detected_sim_mono

#protons_detected_sim_mono = np.array([920, 508, 278, 161, 83, 51])- muons_detected_sim_mono # 2.5


M_n_full = 8771
P_n_full = 7467508-M_n_full
cuts = np.array([30, 40, 50, 60, 70, 80]) #, 90])
muons_detected_full = np.array([8695, 8686, 8634, 7130, 4422, 2434]) #, 4])
protons_detected_full = np.array([185934, 108332, 65925, 41309, 26309, 17058]) - muons_detected_full #, 9]) - muons_detected
# PRECISION RECALL
TP_full = muons_detected_full
FP_full = protons_detected_full
TN_full = P_n_full - protons_detected_full
FN_full = M_n_full - muons_detected_full
precision_full = TP_full/(TP_full+FP_full)
recall_full = TP_full/(TP_full+FN_full)



# PRECISION RECALL
TP = muons_detected
FP = protons_detected
TN = P_n - protons_detected
FN = M_n - muons_detected
precision = TP/(TP+FP)
recall = TP/(TP+FN)


TP_sim = muons_detected_sim
FP_sim = protons_detected_sim
TN_sim = P_n_sim - protons_detected_sim
FN_sim = M_n_sim - muons_detected_sim
precision_sim = TP_sim/(TP_sim+FP_sim)
recall_sim = TP_sim/(TP_sim+FN_sim)


TP_sim_mono = muons_detected_sim_mono
FP_sim_mono = protons_detected_sim_mono
TN_sim_mono = P_n_sim_mono - protons_detected_sim_mono
FN_sim_mono = M_n_sim_mono - muons_detected_sim_mono
precision_sim_mono = TP_sim_mono/(TP_sim_mono+FP_sim_mono)
recall_sim_mono = TP_sim_mono/(TP_sim_mono+FN_sim_mono)

print("precision = ", precision)
print("recal = ", recall)
#fig = plt.figure(figsize = (10,9))
_, ax = plt.subplots(2,1,figsize=(12, 12))

ax[0].plot(cuts, precision, label = 'Observational Data')
#ax[0].plot(cuts, precision_sim, label = 'Stereo Simulations')
ax[0].plot(cuts_sim_mono, precision_sim_mono, label = 'Mono Simulations')
ax[0].plot(cuts, precision_full, label = 'Full Obs')
ax[0].grid()
ax[0].legend()
ax[0].set_ylabel("precision")
ax[1].set_xlabel("cuts (min number of pixels)")
ax[1].plot(cuts, recall, label = 'Observational Data')
#ax[1].plot(cuts, recall_sim, label = 'Stereo Simulations')
ax[1].plot(cuts_sim_mono, recall_sim_mono, label = 'Mono Simulations')
ax[1].plot(cuts, recall_full, label = 'Full Obs')
ax[1].set_ylabel("recall")
ax[1].grid()
ax[1].legend()

In [ ]:
np.transpose(np.array(numsamples*[bad_pixels])).shape

In [ ]:
test3 = [1,2,3]
np.array(3*test3)

In [ ]:
len(numsamples*bad_pixels)

In [ ]:
bad_pixels.shape

In [ ]:
test1.shape

In [ ]:
10* test1

In [ ]:
(~bad_waveform).shape

In [ ]:
bad_waveform.shape

In [ ]:
len(~bad_waveform[~bad_waveform==0])

In [ ]:
test2

In [ ]:
event.r1.tel[telescope_id].waveform.shape

In [ ]:
np.rad2deg(0.0035)

In [ ]:
0.08 * 1855

In [ ]:
print(all([1<3, 4<2, 10<20]))

In [ ]:
print(event_id)

In [ ]:
import numpy as np
a = np.random.randn(5).astype(np.uint64)
b = np.random.randn(5).astype(np.uint32)
c = np.float64(3.6).astype(np.uint64)
d = np.float64(3.6).astype(np.uint32)

In [ ]:
test = np.array([[[1],[2]],[[3], [4]]])

In [ ]:
test.shape

In [ ]:
test[0,0,0]